# OpenMined Duet Authority

This is replicating the logic which should ideally be hosted as a Full Stack Application. In particular the [OMAuthorityAgent.py](./src/OMAuthorityAgent.py) class has been copied and slightly edited from this Full Stack Application. The Full Stack App code can be found at this repo - https://github.com/wip-abramson/fpc-om-authority-aries-application

It is hopefully running here - http://139.162.224.50/

### Imports

In [ ]:
from aries_cloudcontroller import AriesAgentController

from src.OMAuthorityAgent import OMAuthorityAgent

import os
from termcolor import colored

### Initialise the Agent Controller

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(
    f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}"
)
agent_controller = AriesAgentController(admin_url, api_key)

### Start a Webhook Server

In [ ]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

## Store Issuing Schema and Cred Def Identifiers

This is helpful as it means as long as you run `./manage.sh stop` to stop the containers you will not need to reinitialise your agent. State will be persisted and the values you copied below will still be valid.

In [ ]:
# PASTE these two variables from 0 - om_authority_init.ipynb
data_scientist_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Scientist:0.0.1"
data_owner_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OM Data Owner:0.0.1"

In [ ]:
# PASTE these two variables from the end of "0 - om_authority_init.ipynb"
data_scientist_cred_def_id = "HJQ3unDAEknf3hk8sLmTKL:3:CL:188831:default"
data_owner_cred_def_id = "HJQ3unDAEknf3hk8sLmTKL:3:CL:188835:default"

In [ ]:
scientist_config = {
    "schema_id": data_scientist_schema_id,
    "cred_def_id": data_scientist_cred_def_id,
}

owner_config = {
    "schema_id": data_owner_schema_id,
    "cred_def_id": data_owner_cred_def_id,
}

## Initialise the OMAuthority Agent

By passing in the configuration jsons for scientist and data owner you identify the schema and cred defs that class will issue to the respective actors.

In [ ]:
om_authority_agent = OMAuthorityAgent(agent_controller, owner_config, scientist_config)

## Optional: Authentiate Against Agent Using OM PKI Course Credential

This is used in the Full Stack Application as a "login" before you can generate connections for the data scientist or owner. The Full Stack App then uses the connection_id to authenticate future api requests, only allowing them for those connections which have succesfully responded to the proof.

### First Configure the Auth Policy

This identifies the set of attributes and constraints under which a proof must be produced. The below cell identifies the OM PKI Course credential you should have issued to your mobile earlier in the course.

**Note: If you have not got this credential you can skip this part.** Or alternatively you could attempt to use combine the recipes to issue yourself a credential against this schema. Or if you are feeling adventurous define and issue a new schema and update the below authentication policy accordingly.

In [ ]:
pki_schema_id = "Sgg1wREgfEwbEPCQn9xEuE:2:OpenMined PKI Course:0.0.1"

req_attrs = [
    {"name": "name", "restrictions": [{"schema_id": pki_schema_id}]},
]

indy_proof_request = {
    "name": "Proof of PKI Course",
    "version": "1.0",
    "requested_attributes": {
        # They must follow this uuid pattern
        f"0_{req_attr['name']}_uuid": req_attr
        for req_attr in req_attrs
    },
    # Predicates allow us to specify range proofs or set membership on attributes. For example greater than 10.
    # We will ignore these for now.
    "requested_predicates": {},
}

om_authority_agent.set_client_auth_policy(indy_proof_request)

### Get Client Invitation and Display as QR Code.

You will need to scan this QRCode using the mobile wallet containing a OpenMined PKI Course Credential

In [ ]:
client_invite_response = om_authority_agent.client_invitation()
## URL For the invitation so it can be displayed as a QRCode
invitation_url = client_invite_response["invite_url"]
## Connection ID used by OM Authority to identify the agent that responds to this invite
client_connection_id = client_invite_response["connection_id"]
print(client_invite_response)

In [ ]:
import qrcode

# Link for connection invitation

# Creating an instance of qrcode
qr = qrcode.QRCode(version=1, box_size=5, border=5)
qr.add_data(invitation_url)
qr.make(fit=True)
img = qr.make_image(fill="black", back_color="white")
img

### Check Connection ID Trusted List

This would be the connection_id used to authenticate a client application. By sending the connection_id with every api request and checking with the agent if this connection_id is "trusted" we have created a (very hacky) user authentication system.

Work is currently ongoing to use the controller the OpenID Connect protocol.

In [ ]:
is_trusted = om_authority_agent.client_connection_trusted(client_connection_id)
print("Is Client Trusted? ", is_trusted)

## Issue Data Owner Credential

Before you can issue a credential you must first establish a connection across which the credential will be issued.

### What Attributes Are You Issuing

The Full Stack Application allows an authenticated user to submit attributes in a form which are then issued as part of a Data Scientist credential. 

In [ ]:
domain = "Healthcare"
name = "Some Hospital"

### Create Data Owner Invitation

This invite would be returned to the frontend. Where it can then be shared with the Data Owner by copying across to the relevant notebook. 

Once the invitation has been accepted and becomes active, the OM Authority agent will automatically issue a Data Owner credential with the attributes passed to it when the invitation was created. See `src/OMAuthorityAgent.py` for the code.

In [ ]:
invitation = om_authority_agent.data_owner_invitation(name, domain)
## Copy this to the Data Owner notebook
print(invitation)

## Issue Data Scientist Credential

### What Attributes Are You Issuing

The Full Stack Application allows an authenticated user to submit attributes in a form which are then issued as part of a Data Scientist credential. 

In [ ]:
scope = "Healthcare"
name = "Some Healthcare Research Org"

### Create Data Scientist Invitation

This invite would be returned to the frontend. Where it can then be shared with the Data Scientist by copying across to the relevant notebook. 

Once the invitation has been accepted and becomes active, the OM Authority agent will automatically issue a Data Scientist credential with the attributes passed to it when the invitation was created. See `src/OMAuthorityAgent.py` for the code.

In [ ]:
invitation = om_authority_agent.data_scientist_invitation(name, scope)
## Copy this to the Data Scientist notebook
print(invitation)

## Congratulations
Both the Data Scientist and Data Owner "notebook applications" should now have the necessary credentials to authenticate each other before establishing a Duet session.

Be sure to check out the [Full Stack Application for this notebook](https://github.com/wip-abramson/fpc-om-authority-aries-application). Hopefully it is hosted somewhere so you can get a feel for what that user experience might be like. 

Much much more work to be done on that front though!

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()